In [73]:
import pandas as pd

pl_2023_summary = pd.read_csv('premier_league_2023_summary.csv')
pl_2022_summary = pd.read_csv('premier_league_2022_summary.csv')
pl_2021_summary = pd.read_csv('premier_league_2021_summary.csv')
pl_2020_summary = pd.read_csv('premier_league_2020_summary.csv')
pl_2019_summary = pd.read_csv('premier_league_2019_summary.csv')

pl_summary = pd.concat([pl_2023_summary, pl_2022_summary, 
                        pl_2021_summary, pl_2020_summary,
                        pl_2019_summary], axis=0)

pl_summary

,Unnamed: 0,League,Match_Date,Matchweek,Home_Team,Home_Formation,Home_Score,Home_xG,Home_Goals,Home_Yellow_Cards,...,SCA_SCA,GCA_SCA,Cmp_Passes,Att_Passes,Cmp_percent_Passes,PrgP_Passes,Carries_Carries,PrgC_Carries,Att_Take_Ons,Succ_Take_Ons
0,1,Premier League,2022-08-05,Premier League (Matchweek 1),Crystal Palace,4-2-3-1,0,1.2,NaN,1,...,18,0,514,609,84.4,34,524,20,29,16
1,2,Premier League,2022-08-05,Premier League (Matchweek 1),Crystal Palace,4-2-3-1,0,1.2,NaN,1,...,18,2,378,469,80.6,40,359,15,17,13
2,3,Premier League,2022-08-06,Premier League (Matchweek 1),Fulham,4-2-3-1,2,1.2,Aleksandar Mitrović · 32&rsquor; Aleksandar Mi...,2,...,14,4,203,340,59.7,31,233,10,11,5
3,4,Premier League,2022-08-06,Premier League (Matchweek 1),Fulham,4-2-3-1,2,1.2,Aleksandar Mitrović · 32&rsquor; Aleksandar Mi...,2,...,20,4,514,675,76.1,53,418,11,17,4
4,5,Premier League,2022-08-06,Premier League (Matchweek 1),Tottenham Hotspur,3-4-3,4,1.5,Ryan Sessegnon · 21&rsquor; Eric Dier · 31&rsq...,3,...,33,6,501,604,82.9,57,303,19,13,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,756,Premier League,2018-10-20,Premier League (Matchweek 9),Huddersfield Town,3-5-1-1,0,0.7,NaN,0,...,22,2,507,638,79.5,54,483,16,16,8
756,757,Premier League,2018-10-21,Premier League (Matchweek 9),Everton,4-2-3-1,2,1.8,Dominic Calvert-Lewin · 87&rsquor; Cenk Tosun ...,2,...,37,3,451,561,80.4,60,437,26,18,8
757,758,Premier League,2018-10-21,Premier League (Matchweek 9),Everton,4-2-3-1,2,1.8,Dominic Calvert-Lewin · 87&rsquor; Cenk Tosun ...,2,...,11,0,272,384,70.8,31,197,18,12,6
758,759,Premier League,2018-10-22,Premier League (Matchweek 9),Arsenal,4-2-3-1,3,2.7,Mesut Özil · 45&rsquor; Pierre-Emerick Aubamey...,2,...,36,6,633,746,84.9,60,638,25,16,12


Chọn những colume cần thiết để build model

#Các chỉ số dưới đây trong tập data là thuộc về một team (tổng tất cả những đóng góp của các cầu thủ) trong một match<br>

Sh: số cú sút<br>
SoT: số cú sút trúng đích (<= Sh)<br>
Touches: chạm bóng<br>
Tkl: tackle, tắc bóng<br>
Int: interception, đánh chặn<br>
Blocks: chặn bóng<br>
<br><br>
#xG<br>
xG_Expected: goals mong đợi (bao gồm cả penalty)<br>
npxG_Expected: goals mong đợi (ko bao gồm penalty)<br>
<br><br>
#shot_creating_actions<br>
SCA_SCA: số lần tạo hành động có thể sút<br>
GCA_SCA: số lần tạo hành động có thể ghi bàn<br>
<br><br>
#passes<br>
Cmp_Passes: số đường chuyền thành công<br>
Att_Passes: số đường chuyền đã thực hiện (>= Cmp_Passes)<br>
Cmp_percent_Passes: tỉ lệ chuyền thành công trên tổng chuyền<br>
PrgP_Passes: progressive passes, chuyền tiến bộ, số đường chuyền tạo được cơ hội<br>
<br><br>
#carries<br>
Carries_Carries: thời gian cầm bóng trên chân<br>
PrgC_Carries: số lần cầm bóng tạo được cơ hội<br>
<br><br>
#take_ons<br>
Att_Take_Ons: số lần tiếp cận hậu vệ đối phương<br>
Succ_Take_Ons: số lần vượt qua hậu vệ đối phương thành công (<= Att_Take_Ons)<br>


In [74]:
pl = pl_summary.loc[:,['Match_Date', 
                                 'Home_Team', 'Home_Formation','Home_Score', 'Home_Yellow_Cards', 
                                 'Home_Red_Cards',
                                 'Away_Team', 'Away_Formation','Away_Score', 'Away_Yellow_Cards', 
                                 'Away_Red_Cards',
                                 'Sh','SoT','Touches','Tkl','Int','Blocks',
                                 'xG_Expected','npxG_Expected',
                                 'SCA_SCA', 'GCA_SCA',
                                 'Cmp_Passes', 'Att_Passes', 'Cmp_percent_Passes', 'PrgP_Passes', 
                                 'Carries_Carries', 'PrgC_Carries', 
                                 'Att_Take_Ons', 'Succ_Take_Ons']]
pl.head()

,Match_Date,Home_Team,Home_Formation,Home_Score,Home_Yellow_Cards,Home_Red_Cards,Away_Team,Away_Formation,Away_Score,Away_Yellow_Cards,...,SCA_SCA,GCA_SCA,Cmp_Passes,Att_Passes,Cmp_percent_Passes,PrgP_Passes,Carries_Carries,PrgC_Carries,Att_Take_Ons,Succ_Take_Ons
0,2022-08-05,Crystal Palace,4-2-3-1,0,1,0,Arsenal,4-3-3,2,2,...,18,0,514,609,84.4,34,524,20,29,16
1,2022-08-05,Crystal Palace,4-2-3-1,0,1,0,Arsenal,4-3-3,2,2,...,18,2,378,469,80.6,40,359,15,17,13
2,2022-08-06,Fulham,4-2-3-1,2,2,0,Liverpool,4-3-3,2,0,...,14,4,203,340,59.7,31,233,10,11,5
3,2022-08-06,Fulham,4-2-3-1,2,2,0,Liverpool,4-3-3,2,0,...,20,4,514,675,76.1,53,418,11,17,4
4,2022-08-06,Tottenham Hotspur,3-4-3,4,3,0,Southampton,5-3-2,1,0,...,33,6,501,604,82.9,57,303,19,13,7


Đưa về một row trong data thành 1 match (vì mặc định 2 row trong data là 1 match)

In [75]:
pl_home = pd.DataFrame()
pl_home = pl.iloc[0:len(pl):2]

pl_away = pd.DataFrame()
pl_away = pl.iloc[1:len(pl):2]

Đổi tên columns

In [76]:
pl_home.columns = ['Match_Date', 
                        'Home_Team', 'Home_Formation', 'Home_Score', 'Home_Yellow_Cards', 
                        'Home_Red_Cards',
                        'Away_Team', 'Away_Formation', 'Away_Score', 'Away_Yellow_Cards', 
                        'Away_Red_Cards',
                        'Home_Sh','Home_SoT','Home_Touches','Home_Tkl','Home_Int','Home_Blocks',
                        'Home_xG_Expected','Home_npxG_Expected',
                        'Home_SCA_SCA', 'Home_GCA_SCA',
                        'Home_Cmp_Passes', 'Home_Att_Passes', 'Home_Cmp_percent_Passes', 'Home_PrgP_Passes', 
                        'Home_Carries_Carries', 'Home_PrgC_Carries', 
                        'Home_Att_Take_Ons', 'Home_Succ_Take_Ons']

pl_home.head()


,Match_Date,Home_Team,Home_Formation,Home_Score,Home_Yellow_Cards,Home_Red_Cards,Away_Team,Away_Formation,Away_Score,Away_Yellow_Cards,...,Home_SCA_SCA,Home_GCA_SCA,Home_Cmp_Passes,Home_Att_Passes,Home_Cmp_percent_Passes,Home_PrgP_Passes,Home_Carries_Carries,Home_PrgC_Carries,Home_Att_Take_Ons,Home_Succ_Take_Ons
0,2022-08-05,Crystal Palace,4-2-3-1,0,1,0,Arsenal,4-3-3,2,2,...,18,0,514,609,84.4,34,524,20,29,16
2,2022-08-06,Fulham,4-2-3-1,2,2,0,Liverpool,4-3-3,2,0,...,14,4,203,340,59.7,31,233,10,11,5
4,2022-08-06,Tottenham Hotspur,3-4-3,4,3,0,Southampton,5-3-2,1,0,...,33,6,501,604,82.9,57,303,19,13,7
6,2022-08-06,Newcastle United,4-3-3,2,0,0,Nottingham Forest,3-4-1-2,0,3,...,45,4,415,526,78.9,69,319,27,22,12
8,2022-08-06,Leeds United,4-2-3-1,2,2,0,Wolverhampton Wanderers,4-2-3-1,1,0,...,18,0,295,399,73.9,37,288,15,14,7


In [77]:
pl_away.columns = ['Match_Date', 
                        'Home_Team', 'Home_Formation', 'Home_Score', 'Home_Yellow_Cards', 
                        'Home_Red_Cards',
                        'Away_Team', 'Away_Formation', 'Away_Score', 'Away_Yellow_Cards', 
                        'Away_Red_Cards',
                        'Away_Sh','Away_SoT','Away_Touches','Away_Tkl','Away_Int','Away_Blocks',
                        'Away_xG_Expected','Away_npxG_Expected',
                        'Away_SCA_SCA', 'Away_GCA_SCA',
                        'Away_Cmp_Passes', 'Away_Att_Passes', 'Away_Cmp_percent_Passes', 'Away_PrgP_Passes', 
                        'Away_Carries_Carries', 'Away_PrgC_Carries', 
                        'Away_Att_Take_Ons', 'Away_Succ_Take_Ons']

pl_away.head()


,Match_Date,Home_Team,Home_Formation,Home_Score,Home_Yellow_Cards,Home_Red_Cards,Away_Team,Away_Formation,Away_Score,Away_Yellow_Cards,...,Away_SCA_SCA,Away_GCA_SCA,Away_Cmp_Passes,Away_Att_Passes,Away_Cmp_percent_Passes,Away_PrgP_Passes,Away_Carries_Carries,Away_PrgC_Carries,Away_Att_Take_Ons,Away_Succ_Take_Ons
1,2022-08-05,Crystal Palace,4-2-3-1,0,1,0,Arsenal,4-3-3,2,2,...,18,2,378,469,80.6,40,359,15,17,13
3,2022-08-06,Fulham,4-2-3-1,2,2,0,Liverpool,4-3-3,2,0,...,20,4,514,675,76.1,53,418,11,17,4
5,2022-08-06,Tottenham Hotspur,3-4-3,4,3,0,Southampton,5-3-2,1,0,...,17,2,351,445,78.9,31,299,12,18,9
7,2022-08-06,Newcastle United,4-3-3,2,0,0,Nottingham Forest,3-4-1-2,0,3,...,8,0,243,342,71.1,16,202,12,17,5
9,2022-08-06,Leeds United,4-2-3-1,2,2,0,Wolverhampton Wanderers,4-2-3-1,1,0,...,27,2,502,597,84.1,44,507,20,41,25


Kết hợp home và away để tạo thành data frame với 1 row tương ứng với 1 match

In [78]:
pl_merge = pd.merge(pl_home, pl_away, on=['Match_Date', 
                                 'Home_Team', 'Home_Formation', 'Home_Score', 'Home_Yellow_Cards', 
                                 'Home_Red_Cards',
                                 'Away_Team', 'Away_Formation', 'Away_Score', 'Away_Yellow_Cards', 
                                 'Away_Red_Cards'], how='inner')
pl_merge

,Match_Date,Home_Team,Home_Formation,Home_Score,Home_Yellow_Cards,Home_Red_Cards,Away_Team,Away_Formation,Away_Score,Away_Yellow_Cards,...,Away_SCA_SCA,Away_GCA_SCA,Away_Cmp_Passes,Away_Att_Passes,Away_Cmp_percent_Passes,Away_PrgP_Passes,Away_Carries_Carries,Away_PrgC_Carries,Away_Att_Take_Ons,Away_Succ_Take_Ons
0,2022-08-05,Crystal Palace,4-2-3-1,0,1,0,Arsenal,4-3-3,2,2,...,18,2,378,469,80.6,40,359,15,17,13
1,2022-08-06,Fulham,4-2-3-1,2,2,0,Liverpool,4-3-3,2,0,...,20,4,514,675,76.1,53,418,11,17,4
2,2022-08-06,Tottenham Hotspur,3-4-3,4,3,0,Southampton,5-3-2,1,0,...,17,2,351,445,78.9,31,299,12,18,9
3,2022-08-06,Newcastle United,4-3-3,2,0,0,Nottingham Forest,3-4-1-2,0,3,...,8,0,243,342,71.1,16,202,12,17,5
4,2022-08-06,Leeds United,4-2-3-1,2,2,0,Wolverhampton Wanderers,4-2-3-1,1,0,...,27,2,502,597,84.1,44,507,20,41,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,2018-10-20,Wolverhampton Wanderers,3-4-3,0,3,0,Watford,4-2-2-2,2,1,...,17,4,482,576,83.7,37,430,13,17,9
1896,2018-10-20,Cardiff City,4-4-1-1,4,3,0,Fulham,4-3-3,2,3,...,18,4,390,510,76.5,35,262,5,4,4
1897,2018-10-20,Huddersfield Town,3-5-1-1,0,0,0,Liverpool,4-3-3,1,2,...,22,2,507,638,79.5,54,483,16,16,8
1898,2018-10-21,Everton,4-2-3-1,2,2,0,Crystal Palace,4-4-2,0,1,...,11,0,272,384,70.8,31,197,18,12,6


In [79]:
pl_matches = pl_merge.loc[:,['Match_Date', 
                                 'Home_Team', 
                                 'Home_Sh','Home_SoT','Home_Touches','Home_Tkl','Home_Int','Home_Blocks',
                                 'Home_xG_Expected','Home_npxG_Expected',
                                 'Home_SCA_SCA', 'Home_GCA_SCA',
                                 'Home_Cmp_Passes', 'Home_Att_Passes', 'Home_Cmp_percent_Passes', 'Home_PrgP_Passes', 
                                 'Home_Carries_Carries', 'Home_PrgC_Carries', 
                                 'Home_Att_Take_Ons', 'Home_Succ_Take_Ons',
                                 'Away_Team',
                                 'Away_Sh','Away_SoT','Away_Touches','Away_Tkl','Away_Int','Away_Blocks',
                                 'Away_xG_Expected','Away_npxG_Expected',
                                 'Away_SCA_SCA', 'Away_GCA_SCA',
                                 'Away_Cmp_Passes', 'Away_Att_Passes', 'Away_Cmp_percent_Passes', 'Away_PrgP_Passes', 
                                 'Away_Carries_Carries', 'Away_PrgC_Carries', 
                                 'Away_Att_Take_Ons', 'Away_Succ_Take_Ons',
                                 'Home_Score', 'Away_Score']]

pl_matches

,Match_Date,Home_Team,Home_Sh,Home_SoT,Home_Touches,Home_Tkl,Home_Int,Home_Blocks,Home_xG_Expected,Home_npxG_Expected,...,Away_Cmp_Passes,Away_Att_Passes,Away_Cmp_percent_Passes,Away_PrgP_Passes,Away_Carries_Carries,Away_PrgC_Carries,Away_Att_Take_Ons,Away_Succ_Take_Ons,Home_Score,Away_Score
0,2022-08-05,Crystal Palace,10,2,726,18,8,10,1.2,1.2,...,378,469,80.6,40,359,15,17,13,0,2
1,2022-08-06,Fulham,8,2,474,24,10,9,1.2,0.5,...,514,675,76.1,53,418,11,17,4,2,2
2,2022-08-06,Tottenham Hotspur,18,8,709,24,3,12,1.6,1.6,...,351,445,78.9,31,299,12,18,9,4,1
3,2022-08-06,Newcastle United,23,9,634,19,11,8,1.7,1.7,...,243,342,71.1,16,202,12,17,5,2,0
4,2022-08-06,Leeds United,12,4,515,21,14,13,0.8,0.8,...,502,597,84.1,44,507,20,41,25,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,2018-10-20,Wolverhampton Wanderers,10,1,557,22,11,12,0.7,0.7,...,482,576,83.7,37,430,13,17,9,0,2
1896,2018-10-20,Cardiff City,22,5,478,12,14,10,1.5,1.5,...,390,510,76.5,35,262,5,4,4,4,2
1897,2018-10-20,Huddersfield Town,13,1,705,20,12,19,0.7,0.7,...,507,638,79.5,54,483,16,16,8,0,1
1898,2018-10-21,Everton,20,4,692,21,11,6,1.8,1.8,...,272,384,70.8,31,197,18,12,6,2,0


Xuất ra CSV

In [83]:
pl_matches.to_csv('pl_matches.csv', index=False)